# Initialize

In [ ]:
# %matplotlib 
%load_ext autoreload
%autoreload 2

In [ ]:
"""
Investigate the behavior of connected particles through statistical tests based on a periodogram
"""

# try:
#     bl_has_run
#
# except Exception:
#     %matplotlib
#     %load_ext autoreload
#     %autoreload 2
#     bl_has_run = True

import cProfile
import os
import pickle
import pstats
import re

import matplotlib.pyplot as plt
import numpy as np
from numpy import arctan, cos, log, pi, sin
from scipy.fftpack import fft
from scipy.optimize import minimize, root_scalar
# eqn(sol.root, sol.root / (alpha + 1)) / eqn(sol.root, D_interval[1])
from scipy.special import gammaln
from tqdm import tqdm
import logging

from likelihood import (estimate_sigma2_matrix,
                        get_ln_likelihood_func_2_particles_x_link,
                        get_ln_likelihood_func_no_link, get_MLE,
                        get_sigma2_matrix_func)
#  Load and plot data of the test result as a function of the link strength
from plot import (plot_angle_dependence, plot_diffusivity_dependence,
                  plot_link_strength_dependence, plot_localization_dependence,
                  plot_periodogram, contour_plot_free_dumbbell_same_D,
                 contour_plot_localized_eta12_v_eta1_eta2, contour_plot_localized_eta12_v_gamma, 
                  contour_plot_localized_eta12_v_eta_ratio, contour_plot_localized_gamma_v_eta,
                 contour_plot_localized_gamma_v_eta_ratio, contour_plot_localized_eta12_v_M, contour_plot_localized_gamma_v_M, 
                  contour_plot_localized_eta12_v_eta)
from simulate import simulate_2_confined_particles_with_fixed_angle_bond
from support import (calculate_min_number_of_tries_with_a_binomial_model,
                     get_rotation_matrix, locally_rotate_a_vector)
from EnergyTransferResults import EnergyTransferResults

arguments_file = 'arguments.dat'

def clear_arguments_file(cluster):
    if cluster: 
        try:    
            os.unlink(arguments_file)
        except OSError:
            pass

#  Constants
D2 = 0.4  # um^2/s
D1 = 5 * D2   # um^2/s; 0.4
n1 = 2e3
n2 = 2e3
n12 = 10 * n2  # s^{-1}. Somehting interesting happens between [1e-9; 1e-6]
N = 101  # required number of points in a trajectory
dt = 0.05  # s 0.3
gamma = 1e-8    # viscous drag, in kg/s
L = 0.5
angle = 0
trial = 0   # the trial number




k1, k2, k12 = np.array([n1, n2, n12]) * gamma
T = dt * (N - 1)  # s
M = N - 1

true_parameters = {name: val for name, val in zip(
    ('D1 D2 n1 n2 n12 gamma T dt angle L trial M'.split()),
    (D1, D2, n1, n2, n12, gamma, T, dt, angle, L, trial, M))}

In [ ]:
# exact:
# a0eee63cfa1c82304d1ee740c13c900d
# 0.4 0.4 0 0 0.19999999999999998 0.05 0 0.014142135623730954 0 100 free_same_D 0

# arg_string:
# a0eee63cfa1c82304d1ee740c13c900d
# 0.4 0.4 0.0 0.0 0.2 0.05 0 0.0141421 0 100 free_same_D False


## Testing the new free Hookean dumbbell model

In [ ]:
recalculate_trajectory = 0
# recalculate_BF = 1
cluster = 1

l0_range=[3e-1, 1e4]
eta_range=[1e-2, 1e3]

clear_arguments_file(cluster)
    
#
lgbfvals = contour_plot_free_dumbbell_same_D(trials=100, Ms=[1000], l0_range=l0_range, eta_range=eta_range, 
                                             verbose=False, recalculate_trajectory=recalculate_trajectory,
                                      cluster=cluster, dt = 0.5)

# l0_range=[1e-1, 1e4], eta_range=[1e-5, 1e3]
# l0_range=[1e-1, 1e3], eta_range=[1e-2, 1e2]

In [ ]:
recalculate_trajectory = 0
# recalculate_BF = 1
cluster = 1

l0_range=[1e-1, 1e5]
eta_range=[1e-2, 200]

clear_arguments_file(cluster)
    
#
lgbfvals = contour_plot_free_dumbbell_same_D(trials=10, Ms=[200, 1000], l0_range=l0_range, eta_range=eta_range, 
                                             verbose=False, recalculate_trajectory=recalculate_trajectory,
                                      cluster=cluster, dt = 0.05)

# l0_range=[1e-1, 1e4], eta_range=[1e-5, 1e3]
# l0_range=[1e-1, 1e3], eta_range=[1e-2, 1e2]

# 2D plots for the localized system

## Initialization cell

In [ ]:
recalculate_trajectory = 0
cluster = 1

clear_arguments_file(cluster)
group_calculation = True
trials_low_res = 50
trials_high_res = 200
Ms = (1000, )
gamma_range = (1e-2, 100)
eta12_range = (1e-3, 100)
eta_range =  (1e-2, 100)
# eta_ratio_range= (1e-3, 1e3)
M_range= list(range(1,10)) +  list(range(10,100,10)) + list(range(100, 1000, 100))


if recalculate_trajectory:
    logging.warning("Trajectories will be recalculated!")

## The cells below investigate the dependence on various combinations of the 5 parameters of the model

## Link strength and diffusivity ratio $\lg B(\eta_{12}, \gamma \equiv D_1/D_2)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_eta12_v_gamma(
    trials=trials_low_res,
    verbose=False,
    recalculate_trajectory=recalculate_trajectory,
    cluster=cluster,
    Ms= Ms,
    eta12_range=eta12_range,
    gamma_range=gamma_range,
    dt=0.05,
    angle=0,
)

## Link strength and localization $\lg B(\eta_{12}, \eta_1)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_eta12_v_eta(
    var = 'eta1',
    trials=trials_high_res,
    verbose=False,
    recalculate_trajectory=recalculate_trajectory,
    cluster=cluster,
    Ms= Ms,
    eta12_range=eta12_range,
    eta_range=eta_range,
    dt=0.05,
    angle=0,
)

## Link strength and localization $\lg B(\eta_{12}, \eta_2)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_eta12_v_eta(
    var = 'eta2',
    trials=trials_low_res,
    verbose=False,
    recalculate_trajectory=recalculate_trajectory,
    cluster=cluster,
    Ms= Ms,
    eta12_range=eta12_range,
    eta_range=eta_range,
    dt=0.05,
    angle=0,
)

## Link strength and trajectory length $\lg B(\eta_{12}, M)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_eta12_v_M(
        trials=trials_low_res,
        verbose=False,
        recalculate_trajectory=recalculate_trajectory,
        cluster=cluster,
        eta12_range=eta12_range,
        M_range=M_range,
        dt=0.05,
        angle=0,
)

## Diffusivity ratio and localization $\lg B(\gamma, \eta_1)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_gamma_v_eta(
        var = 'eta1',
        trials=trials_high_res,
        verbose=False,
        recalculate_trajectory=recalculate_trajectory,
        cluster=cluster,
        Ms= Ms,
        gamma_range=gamma_range,
        eta_range=eta_range,
        dt=0.05,
        angle=0,
)

## Diffusivity ratio and localization $\lg B(\gamma, \eta_2)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_gamma_v_eta(
        var = 'eta2',
        trials=trials_low_res,
        verbose=False,
        recalculate_trajectory=recalculate_trajectory,
        cluster=cluster,
        Ms= Ms,
        gamma_range=gamma_range,
        eta_range=eta_range,
        dt=0.05,
        angle=0,
)

## Diffusivity ratio and trajectory length $\lg B(\gamma, M)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_gamma_v_M(
        trials=trials_low_res,
        verbose=False,
        recalculate_trajectory=recalculate_trajectory,
        cluster=cluster,
        gamma_range=gamma_range,
        M_range=M_range,
        dt=0.05,
        angle=0,
)

# Test for energy transfer

In [ ]:
from plot import contour_plot_localized_eta12_v_gamma_energy_transfer_triple_test
clear_arguments_file(cluster)
group_calculation = False

## Link strength and diffusivity ratio $\lg B(\eta_{12}, \gamma \equiv D_1/D_2)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_eta12_v_gamma_energy_transfer_triple_test(
    trials=trials_high_res,
    verbose=False,
    recalculate_trajectory=recalculate_trajectory,
    cluster=cluster,
    Ms= Ms,
    eta12_range=eta12_range,
    gamma_range=gamma_range,
    dt=0.05,
    angle=0,
    print_MLE = False,
    mesh_resolution = 2**4 + 1,
)

In [ ]:
help(EnergyTransferResults.__init__)

Class version of energy transfer scheduling

In [ ]:
def update_x(args_dict, x):
    D1 = D2 * x
    args_dict.update({'D1': D1})

def update_y(args_dict, y):
    args_dict.update({'n12': y / dt})    

if not group_calculation:
    clear_arguments_file(cluster)

results_eta12_gamma = EnergyTransferResults(
    trials=trials_high_res,
    resolution=4,
    verbose=False,
    cluster=cluster,
    Ms=Ms,
    x_label=r'$\eta_1 / \eta_2 \equiv n_1 / n_2$',
    x_range=gamma_range,
    update_x=update_x,    
    y_label=r'$\eta_{12}$',
    y_range=eta12_range,
    update_y = update_y,
    dt=0.05,
    angle=0,
    rotation=True,
    print_MLE=False,
    title = 'n1={n1:.2f}, n2={n2:.2f}, D2={D2:.2f},\ndt={dt}, L0={L0:.2f}',
    figname_base='localized_gamma_v_eta12_energy_transfer',
)
    

In [ ]:
results_eta12_gamma.run()

## Link strength and localization $\lg B(\eta_{12}, \eta_1)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_eta12_v_eta(
    var = 'eta1',
    trials=trials_high_res,
    verbose=False,
    recalculate_trajectory=recalculate_trajectory,
    cluster=cluster,
    Ms= Ms,
    eta12_range=eta12_range,
    eta_range=eta_range,
    dt=0.05,
    angle=0,
)

## Link strength and localization $\lg B(\eta_{12}, \eta_2)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_eta12_v_eta(
    var = 'eta2',
    trials=trials_low_res,
    verbose=False,
    recalculate_trajectory=recalculate_trajectory,
    cluster=cluster,
    Ms= Ms,
    eta12_range=eta12_range,
    eta_range=eta_range,
    dt=0.05,
    angle=0,
)

## Link strength and trajectory length $\lg B(\eta_{12}, M)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_eta12_v_M(
        trials=trials_low_res,
        verbose=False,
        recalculate_trajectory=recalculate_trajectory,
        cluster=cluster,
        eta12_range=eta12_range,
        M_range=M_range,
        dt=0.05,
        angle=0,
)

## Diffusivity ratio and localization $\lg B(\gamma, \eta_1)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_gamma_v_eta(
        var = 'eta1',
        trials=trials_high_res,
        verbose=False,
        recalculate_trajectory=recalculate_trajectory,
        cluster=cluster,
        Ms= Ms,
        gamma_range=gamma_range,
        eta_range=eta_range,
        dt=0.05,
        angle=0,
)

## Diffusivity ratio and localization $\lg B(\gamma, \eta_2)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_gamma_v_eta(
        var = 'eta2',
        trials=trials_low_res,
        verbose=False,
        recalculate_trajectory=recalculate_trajectory,
        cluster=cluster,
        Ms= Ms,
        gamma_range=gamma_range,
        eta_range=eta_range,
        dt=0.05,
        angle=0,
)

## Diffusivity ratio and trajectory length $\lg B(\gamma, M)$

In [ ]:
if not group_calculation:
    clear_arguments_file(cluster)
    
_ = contour_plot_localized_gamma_v_M(
        trials=trials_low_res,
        verbose=False,
        recalculate_trajectory=recalculate_trajectory,
        cluster=cluster,
        gamma_range=gamma_range,
        M_range=M_range,
        dt=0.05,
        angle=0,
)

# Other

## Test the free Hookean model on a trajectory with no link

In [ ]:
recalculate_trajectory = 0
cluster = 1

clear_arguments_file(cluster)
    
#
lgbfvals = plot_free_hookean_length_dependence(trials=50, verbose=False,
                                        recalculate_trajectory=False,
                                        cluster=cluster, Ms=[1000], l0_range=[1e-1, 1e4],
                                        model='free_same_D')

## Localized models

In [ ]:
lg_BF_vals = plot_free_hookean_link_strength_dependence(
    trials=20, n12_range=[1e-1, 1e2], verbose=False, recalculate_trajectory=recalculate_trajectory, 
    cluster=cluster, model = 'free_same_D')

In [ ]:
lg_BF_vals = plot_link_strength_dependence(
    trials=5, n12_range=[1e-1, 1e2], verbose=False, recalculate_trajectory=recalculate_trajectory, recalculate_BF=recalculate_BF, cluster=cluster)

lg_BF_vals = plot_diffusivity_dependence(
    trials=200, D1_range=[1e-2, 5], verbose=False, recalculate_trajectory=recalculate_trajectory, recalculate_BF=recalculate_BF, cluster=cluster)


lg_BF_vals = plot_localization_dependence(
    trials=200, n_range=[0.3, 100], verbose=False, recalculate_trajectory=recalculate_trajectory, recalculate_BF=recalculate_BF, cluster=cluster)

## Angle detection

In [ ]:
recalculate_trajectory = 0
recalculate_BF = 0
cluster = 1
clear_arguments_file(cluster)
        

lg_BF_vals = plot_angle_dependence(
     trials=10, points = 2**6 +1, verbose=False, recalculate_trajectory=recalculate_trajectory,
     recalculate_BF=recalculate_BF, cluster=cluster)

## Dependence on the link strength

In [ ]:
recalculate_trajectory = 0
recalculate_BF = 0
cluster = 1
clear_arguments_file(cluster)

lg_BF_vals = plot_link_strength_dependence(
    trials=10, points = 2**5 + 1, n12_range=[1e-1, 1e2], verbose=False, recalculate_trajectory=recalculate_trajectory,
    recalculate_BF=recalculate_BF, cluster=cluster)

## Dependence on the diffusivity

In [ ]:
recalculate_trajectory = 0
recalculate_BF = 0
cluster = 1
clear_arguments_file(cluster)

plot_diffusivity_dependence(
    trials=10, points = 2**5 + 1, D1_range=[1e-2, 5], verbose=False, recalculate_trajectory=recalculate_trajectory,
    recalculate_BF=recalculate_BF, cluster=cluster)

In [ ]:
# Yomoni
x = range(4, 10)
y1 = [5.23, 6.4, 7.61, 8.96, 10.32, 11.68]
y3=[8.43, 9.25,10.09,12.13,14,15.87]
yall=[15.04, 16.86, 18.73,23.17,27.56,30.26]
yloss = [4.7, 6.3, 7.8, 9.1, 10.3, 11.6]
import matplotlib.pyplot as plt
plt.plot(x,y1, label = 'gain 1')
plt.plot(x,y3, label = 'gain 3')
plt.plot(x,yall, label = 'gain all')
plt.plot(x,yloss, label = 'max. loss')
plt.plot(x,x, label = 'unity')
plt.legend()
# Conclusion: 7 or 8
